In [ ]:
from langchain_community.llms import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
from transformers import pipeline

# Create the LLM -> decide on which device to run model currenlty using cpu hence give message sevice set to run on cpu
pipe = pipeline("text2text-generation", model="google/flan-t5-base")
local_llm = HuggingFacePipeline(pipeline=pipe)

# Create the prompt template + pass input variable to it
prompt = PromptTemplate.from_template("Explain {topic} in simple words.")

# RunnableSequence : It’s not a normal “function”, but a LangChain class that behaves like a function
# Combine them into one "runnable sequence" is is take prompt op and send it to llm 
chain = RunnableSequence(prompt | local_llm)

# Use it like a function
response = chain.invoke({"topic": "Who is Virat Kolhi"})
print(response)


In [1]:
# --- 📦 Imports ---
import transformers
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence

# --- 🧩 Optional: Hide unnecessary Hugging Face logs (like "Device set to use CPU") ---
transformers.utils.logging.set_verbosity_error()

# --- ⚙️ Step 1: Load the Local Model ---
# Model size options:
# "google/flan-t5-small" → light & fast
# "google/flan-t5-base"  → better answers, moderate size (~1GB)
# "google/flan-t5-large" → more accurate, heavy (~3–4GB)
model_name = "google/flan-t5-base"

# Create text2text pipeline (runs on CPU or GPU automatically)
pipe = pipeline(
    task="text2text-generation",
    model=model_name,
    max_new_tokens=200,
    temperature=0.7,    # Controls creativity: 0.1 = factual, 1.0 = creative
)

# Wrap pipeline for LangChain compatibility
local_llm = HuggingFacePipeline(pipeline=pipe)

print("✅ Model loaded successfully!")

# --- 💬 Step 2: Create Prompt Template ---
# Defines how user input (variable) will be inserted into the question RunnableSequence : It’s not a normal “function”, but a LangChain class that behaves like a function
prompt = PromptTemplate.from_template("Explain {topic} in simple and short words.")

# --- 🔗 Step 3: Build a Runnable Sequence ---
# Connects prompt → model (output of prompt goes as input to our local LLM variable)
chain = RunnableSequence(prompt | local_llm)

# --- 🧠 Step 4: Ask a Question (like a function call) ---
query = {"topic": "java?"}
response = chain.invoke(query)

# --- 🖨️ Step 5: Display Result ---
print("\n🧠 AI Answer:\n", response)


c:\Users\LENOVO\OneDrive\Desktop\Langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_15912\3740355864.py:27: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFacePipeline``.
  local_llm = HuggingFacePipeline(pipeline=pipe)


✅ Model loaded successfully!

🧠 AI Answer:
 java bean


In [ ]:
# --- Imports ---
import transformers
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence

# --- Hide logs ---
transformers.utils.logging.set_verbosity_error()

# --- Load model ---
pipe = pipeline("text2text-generation", model="google/flan-t5-base", max_new_tokens=250)
local_llm = HuggingFacePipeline(pipeline=pipe)

# --- Define prompts ---
explain_prompt = PromptTemplate.from_template("Explain {topic} in simple words.")
adv_prompt = PromptTemplate.from_template("Based on this explanation: {explanation}, list 3 advantages of {topic}.")
example_prompt = PromptTemplate.from_template("From the advantages: {advantages}, give one real-world example of {topic}.")

# --- Build sub-chains ---
explain_chain = explain_prompt | local_llm
adv_chain = adv_prompt | local_llm
example_chain = example_prompt | local_llm

# --- Connect sequentially ---
# Step 1 → Step 2 → Step 3
chain = (
    explain_chain
    | (lambda text: {"topic": "Java programming language", "explanation": text})
    | adv_chain
    | (lambda text: {"topic": "Java programming language", "advantages": text})
    | example_chain
)

# --- Run the chain ---
response = chain.invoke({"topic": "Java programming language"})

print("\n🧠 Final AI Output:\n", response)


: 

In [ ]:
# --- 📦 Imports ---
import transformers
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
import gradio as gr

# --- Hide unnecessary logs ---
transformers.utils.logging.set_verbosity_error()

# --- Load Local Model ---
pipe = pipeline("text2text-generation", model="google/flan-t5-base", max_new_tokens=300)
local_llm = HuggingFacePipeline(pipeline=pipe)

# --- Define Prompts ---
explain_prompt = PromptTemplate.from_template("Explain {topic} in simple words.")
adv_prompt = PromptTemplate.from_template("Based on this explanation: {explanation}, list 3 advantages of {topic}.")
example_prompt = PromptTemplate.from_template("From the advantages: {advantages}, give one real-world example of {topic}.")

# --- Create Subchains ---
explain_chain = explain_prompt | local_llm
adv_chain = adv_prompt | local_llm
example_chain = example_prompt | local_llm

# --- Function for Gradio ---
def explain_topic(topic):
    explanation = explain_chain.invoke({"topic": topic})
    advantages = adv_chain.invoke({"topic": topic, "explanation": explanation})
    example = example_chain.invoke({"topic": topic, "advantages": advantages})
    return explanation, advantages, example

# --- Build Gradio Interface ---
demo = gr.Interface(
    fn=explain_topic,
    inputs=gr.Textbox(label="Enter a Topic (e.g., Java, Blockchain, Machine Learning)"),
    outputs=[
        gr.Textbox(label="💬 Explanation"),
        gr.Textbox(label="💡 Advantages"),
        gr.Textbox(label="🧰 Real-world Example"),
    ],
    title="LangChain Multi-Step AI Assistant",
    description="Type any topic, and the AI will explain it, list advantages, and give one real-world example.",
)

# --- Launch the app ---
demo.launch()


: 